In [ ]:
!pip install keras

In [ ]:
%pylab inline

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import cross_val_score, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LassoCV, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import seaborn

In [ ]:
def train_test_split(X, y):
    N = X.shape[0]
    split_size = int(N/5)
    split = int(N - 2*split_size)
    train_X = X[:split]
    train_y = y[:split]
    val_X = X[split:split+split_size]
    val_y = y[split:split+split_size]
    test_X = X[split+split_size:]
    test_y = y[split+split_size:]
    
    return train_X, train_y, val_X, val_y, test_X, test_y

def test_model(model, params, train_X, train_y, test_X, test_y):
    scorer = make_scorer(mean_squared_error, greater_is_better=False) # Use mean squared error as score
    gs = GridSearchCV(model, params, scoring=scorer) # Cross-validation to pick best hyperparameter
    gs.fit(train_X, train_y)
    gs.best_estimator_.fit(train_X, train_y)
    train_pred = gs.best_estimator_.predict(train_X) # Make prediction on training set
    test_pred = gs.best_estimator_.predict(test_X) # Make prediction on test set
    print("Training MSE: ", mean_squared_error(train_y, train_pred))
    print("Test MSE: ", mean_squared_error(test_y, test_pred))
    return gs.best_estimator_

### Tasks

1. Find the best model for predicting the returns of the S&P500
2. Advanced: calculate the returns of your model if it was put into use

### Load Data

#### Data consists of log-returns of the S&P500 on a particular date, with some other indicators that might predict it.

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/danybol/gft_ml_example/master/worked_example/real_data.csv", parse_dates=['date'])
data.head()

In [ ]:
### Put your preprocessing code here




In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)
train_X = train_data.drop(['returns', 'date'], axis=1)
train_y = train_data['returns']
test_X = test_data.drop(['returns', 'date'], axis=1)
test_y = test_data['returns']

In [ ]:
model = Pipeline([('features', None),
                  ('preprocess', None),
                  ('model', None)])


In [ ]:
params = dict(model=[Lasso()], model__alpha=np.logspace(-2, 3), features=[None], preprocess=[None])
simple_model = test_model(model, params, train_X, train_y, test_X, test_y)
test_pred = simple_model.predict(test_X)

In [ ]:
plot(test_y.values[0:10])
plot(test_pred[0:10])